In [3]:
import numpy as np
import pandas as pd
import ta
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
%matplotlib inline

In [52]:
symbols = ['EURUSDmicro','GBPUSDmicro','USDCHFmicro','USDJPYmicro','USDCADmicro']

timeframe = 'D1'

df = pd.read_csv("data/"+ symbols[0]+ timeframe+ ".csv")
df = df[['Date','Open','High','Low','Close']]
df['Date'] = pd.to_datetime(df['Date'])
df['Return'] =   df['Close'].pct_change() # Retorno percentual
df['Alta/Baixa'] = [1 if x >= 0 else 0 for x in df['Return']] # dia de alta ou de baixa

df['A'] = df['High'] / df['Low']   # Razão entre a máxima e mínima do candle

df['B'] = (abs(df['Close'] - df['Open'])) / (df['High'] - df['Low'])  # Razão entre o corpo e a amplitude do candle

sombra1 = (df['High'] - df['Close']) / (df['Open'] - df['Low'])
sombra2 = (df['High'] - df['Open']) / (df['Close'] - df['Low'])

df['C'] = [sombra1.loc[x] if df.loc[x, 'Alta/Baixa'] == 1 else sombra2.loc[x] for x in df.index] # Razão entre a sombra superior e inferior

df['D'] = df['Close'] / df['Close'].shift(1) # Razão entre o fechamento de Hoje e o de x dias anteriores

df['E'] = df['Close'] - df['Close'].rolling(8).mean() # Distância entre o preço de hoje e a média curta

df['F'] = df['Close'] - df['Close'].rolling(80).mean() # Distância entre o preço de hoje e a média Longa

df['G'] = df['Close'].rolling(8).mean() - df['Close'].rolling(80).mean() # Distância entre o preço das duas médias

df['H'] = df['Return'].rolling(5).std() # Volatildiade dos últimos x dias

df['I'] = df['H'].rolling(5).sum() # Volatildiade acumulada dos últimos x dias

df['alvo'] = df['Alta/Baixa'].shift(-1)
df.dropna(inplace=True)

df.head()

,Date,Open,High,Low,Close,Return,Alta/Baixa,A,B,C,D,E,F,G,H,I,alvo
79,2013-04-22,1.30763,1.30823,1.30138,1.30649,0.001142,1,1.005264,0.166423,0.278400,1.001142,-0.001170,-0.009331,-0.008161,0.007745,0.038600,0.0
80,2013-04-23,1.30648,1.30824,1.29715,1.29952,-0.005335,0,1.008550,0.627592,0.742616,0.994665,-0.006848,-0.016064,-0.009217,0.005336,0.037843,1.0
81,2013-04-24,1.29957,1.30325,1.29538,1.30145,0.001485,1,1.006075,0.238882,0.429594,1.001485,-0.003730,-0.014093,-0.010363,0.002904,0.032438,0.0
82,2013-04-25,1.30144,1.30919,1.29875,1.30101,-0.000338,0,1.008038,0.041188,3.429204,0.999662,-0.003879,-0.014461,-0.010582,0.002749,0.026945,1.0
83,2013-04-26,1.30102,1.30461,1.29892,1.30277,0.001353,1,1.004381,0.307557,0.876190,1.001353,-0.000275,-0.012592,-0.012317,0.002887,0.021620,1.0


In [47]:
indicador_bb = ta.volatility.BollingerBands(close = df['Close'], n = 20, ndev = 2, fillna = False)
indicador_rsi = ta.momentum.RSIIndicator(close = df['Close'], n = 14, fillna = False)
indicador_macd = ta.trend.MACD(close = df['Close'], n_slow = 26, n_fast = 12, n_sign = 9, fillna = False)

df['bb'] = indicador_bb.bollinger_pband()
df['bbH'] = indicador_bb.bollinger_hband_indicator() 
df['bbL'] = indicador_bb.bollinger_lband_indicator() 


df['rsi'] = indicador_rsi.rsi()
df['rsiH'] = [1 if x > 70  else 0 for x in indicador_rsi.rsi()]
df['rsiL'] = [1 if x < 30  else 0 for x in indicador_rsi.rsi()]

df['macd'] =  indicador_macd.macd_diff()
df['macdH'] =  [1 if x > 0 else 0 for x in indicador_macd.macd_diff()]
df['macdL'] =  [1 if x < 0 else 0 for x in indicador_macd.macd_diff()]

df.dropna(inplace=True)
df

,Date,Open,High,Low,Close,Return,Alta/Baixa,A,B,C,...,alvo,bb,bbH,bbL,rsi,rsiH,rsiL,macd,macdH,macdL
112,2013-06-06,1.30915,1.33041,1.30737,1.32443,0.011664,1,1.017623,0.663194,3.359551,...,0.0,1.206564,1.0,0.0,67.957459,0,0,0.003605,1,0
113,2013-06-07,1.32445,1.32836,1.31905,1.32169,-0.002069,0,1.007058,0.296455,1.481061,...,1.0,1.032725,1.0,0.0,65.181773,0,0,0.003858,1,0
114,2013-06-10,1.31904,1.32677,1.31761,1.32557,0.002936,1,1.006952,0.712882,0.839161,...,1.0,1.016653,1.0,0.0,67.223340,0,0,0.004061,1,0
115,2013-06-11,1.32556,1.33161,1.32314,1.33122,0.004262,1,1.006401,0.668241,0.161157,...,1.0,1.020220,1.0,0.0,69.983408,0,0,0.004327,1,0
116,2013-06-12,1.33123,1.33581,1.32644,1.33356,0.001758,1,1.007064,0.248666,0.469729,...,1.0,0.977778,0.0,0.0,71.069974,1,0,0.004397,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010,2020-09-10,1.17980,1.19164,1.17953,1.18130,0.000966,1,1.010267,0.123865,38.296296,...,1.0,0.327280,0.0,0.0,52.278835,0,0,-0.002072,0,1
2011,2020-09-11,1.18132,1.18732,1.18093,1.18426,0.002506,1,1.005411,0.460094,7.846154,...,1.0,0.491073,0.0,0.0,55.296504,0,0,-0.001734,0,1
2012,2020-09-14,1.18306,1.18870,1.18305,1.18606,0.001520,1,1.004776,0.530973,264.000000,...,0.0,0.593899,0.0,0.0,57.074148,0,0,-0.001358,0,1
2013,2020-09-15,1.18593,1.18995,1.18389,1.18449,-0.001324,0,1.005119,0.237624,6.700000,...,0.0,0.533149,0.0,0.0,55.019069,0,0,-0.001191,0,1


In [48]:
variaveis = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
X = df[variaveis]
y = df['alvo']
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.7, random_state=42)

In [51]:
from sklearn.tree import DecisionTreeClassifier

classifier_dt = DecisionTreeClassifier(random_state=42, criterion='gini', max_depth=3)
classifier_dt.fit(X, y)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').